# Data and Library Loading

In [2]:
import numpy as np
import pandas as pd
import os

import sklearn.model_selection
import sklearn.metrics
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time

from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import *
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting
from keras import activations
from keras import regularizers
from keras import initializers
from keras import constraints
from sklearn.preprocessing import MinMaxScaler
from numba import jit
from math import log, floor
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from keras.engine import Layer
from keras.engine import InputSpec
from keras.objectives import categorical_crossentropy
from keras.objectives import sparse_categorical_crossentropy
import tensorflow as tf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.style.use('seaborn')
sns.set(font_scale=1)

Using TensorFlow backend.


In [3]:
df_train = pd.read_csv('../input/vsb-power-line-fault-detection/metadata_train.csv')
df_train = df_train.set_index(['id_measurement', 'phase'])

X = np.load("../input/folk-base-neural-network-using-lstm/X.npy")
y = np.load("../input/folk-base-neural-network-using-lstm/y.npy")
features = np.load("../input/folk-base-neural-network-using-lstm/features.npy")

In [4]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [5]:
X.shape, y.shape

((2904, 160, 57), (2904,))

In [6]:
%%time
X, y = augment(X,y)
print(X.shape, y.shape)

(5986, 160, 57) (5986,)
CPU times: user 5.63 s, sys: 16.3 s, total: 21.9 s
Wall time: 22 s


In [7]:
features = np.load('../input/vsb-aug-features/aug_features.npy')
features.shape

(5986, 7)

# Building a RNN Model Architecture

In [8]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [9]:
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [15]:
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)]):
        score = K.eval(matthews_correlation(y_true.astype(np.float64), (y_proba > threshold).astype(np.float64)))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'matthews_correlation': best_score}
    return search_result

In [10]:
# This is NN LSTM Model creation
def model_lstm(input_shape, feat_shape):
    inp = Input(shape=(input_shape[1], input_shape[2],))
    feat = Input(shape=(feat_shape[1],))

    bi_lstm_1 = Bidirectional(CuDNNLSTM(128, return_sequences=True), merge_mode='concat')(inp)
    bi_lstm_2 = Bidirectional(CuDNNGRU(64, return_sequences=True), merge_mode='concat')(bi_lstm_1)
    
    attention = Attention(input_shape[1])(bi_lstm_2)
    
    x = concatenate([attention, feat], axis=1)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[inp, feat], outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[matthews_correlation])
    
    return model


In [12]:
# Here is where the training happens
# First, create a set of indexes of the 5 folds
N_SPLITS = 5
splits = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=2019).split(X, y))
preds_val = []
y_val = []
# Then, iteract with each fold
# If you dont know, enumerate(['a', 'b', 'c']) returns [(0, 'a'), (1, 'b'), (2, 'c')]
for idx, (train_idx, val_idx) in enumerate(splits):
    K.clear_session() # I dont know what it do, but I imagine that it "clear session" :)
    print("Beginning fold {}".format(idx+1))
    # use the indexes to extract the folds in the train and validation data
    train_X, train_feat, train_y, val_X, val_feat, val_y = X[train_idx], features[train_idx], y[train_idx], X[val_idx], features[val_idx], y[val_idx]
    # instantiate the model for this fold
    model = model_lstm(train_X.shape, features.shape)
    # This checkpoint helps to avoid overfitting. It just save the weights of the model if it delivered an
    # validation matthews_correlation greater than the last one.
    ckpt = ModelCheckpoint('weights_{}.h5'.format(idx), save_best_only=True, save_weights_only=True, verbose=1, monitor='val_matthews_correlation', mode='max')
    # Train, train, train
    model.fit([train_X, train_feat], train_y, batch_size=128, epochs=50, validation_data=([val_X, val_feat], val_y), callbacks=[ckpt])
    # loads the best weights saved by the checkpoint
    model.load_weights('weights_{}.h5'.format(idx))
    # Add the predictions of the validation to the list preds_val
    preds_val.append(model.predict([val_X, val_feat], batch_size=512))
    # and the val true y
    y_val.append(val_y)

# concatenates all and prints the shape    
preds_val = np.concatenate(preds_val)[...,0]
y_val = np.concatenate(y_val)
preds_val.shape, y_val.shape


Beginning fold 1
Train on 4788 samples, validate on 1198 samples
Epoch 1/50
4788/4788 [==============================] - 5s 989us/step - loss: 0.3383 - matthews_correlation: 0.0345 - val_loss: 0.1725 - val_matthews_correlation: 0.2639

Epoch 00001: val_matthews_correlation improved from -inf to 0.26387, saving model to weights_0.h5
Epoch 2/50
4788/4788 [==============================] - 2s 457us/step - loss: 0.1467 - matthews_correlation: 0.7269 - val_loss: 0.1195 - val_matthews_correlation: 0.2872

Epoch 00002: val_matthews_correlation improved from 0.26387 to 0.28719, saving model to weights_0.h5
Epoch 3/50
4788/4788 [==============================] - 2s 458us/step - loss: 0.1139 - matthews_correlation: 0.7744 - val_loss: 0.0954 - val_matthews_correlation: 0.3596

Epoch 00003: val_matthews_correlation improved from 0.28719 to 0.35955, saving model to weights_0.h5
Epoch 4/50
4788/4788 [==============================] - 2s 461us/step - loss: 0.0847 - matthews_correlation: 0.8052 - val_

4788/4788 [==============================] - 2s 459us/step - loss: 0.0489 - matthews_correlation: 0.8894 - val_loss: 0.0423 - val_matthews_correlation: 0.5238

Epoch 00034: val_matthews_correlation did not improve from 0.54379
Epoch 35/50
4788/4788 [==============================] - 2s 460us/step - loss: 0.0442 - matthews_correlation: 0.8975 - val_loss: 0.0483 - val_matthews_correlation: 0.4779

Epoch 00035: val_matthews_correlation did not improve from 0.54379
Epoch 36/50
4788/4788 [==============================] - 2s 461us/step - loss: 0.0481 - matthews_correlation: 0.8807 - val_loss: 0.0460 - val_matthews_correlation: 0.5199

Epoch 00036: val_matthews_correlation did not improve from 0.54379
Epoch 37/50
4788/4788 [==============================] - 2s 460us/step - loss: 0.0458 - matthews_correlation: 0.8815 - val_loss: 0.0383 - val_matthews_correlation: 0.5120

Epoch 00037: val_matthews_correlation did not improve from 0.54379
Epoch 38/50
4788/4788 [==============================] -


Epoch 00016: val_matthews_correlation did not improve from 0.54315
Epoch 17/50
4788/4788 [==============================] - 2s 462us/step - loss: 0.0513 - matthews_correlation: 0.8822 - val_loss: 0.0456 - val_matthews_correlation: 0.4916

Epoch 00017: val_matthews_correlation did not improve from 0.54315
Epoch 18/50
4788/4788 [==============================] - 2s 463us/step - loss: 0.0496 - matthews_correlation: 0.8744 - val_loss: 0.0450 - val_matthews_correlation: 0.5197

Epoch 00018: val_matthews_correlation did not improve from 0.54315
Epoch 19/50
4788/4788 [==============================] - 2s 463us/step - loss: 0.0520 - matthews_correlation: 0.8737 - val_loss: 0.0466 - val_matthews_correlation: 0.4889

Epoch 00019: val_matthews_correlation did not improve from 0.54315
Epoch 20/50
4788/4788 [==============================] - 2s 463us/step - loss: 0.0477 - matthews_correlation: 0.8906 - val_loss: 0.0467 - val_matthews_correlation: 0.5176

Epoch 00020: val_matthews_correlation did n

Beginning fold 3
Train on 4789 samples, validate on 1197 samples
Epoch 1/50
4789/4789 [==============================] - 3s 668us/step - loss: 0.3391 - matthews_correlation: 0.0850 - val_loss: 0.2183 - val_matthews_correlation: 0.2348

Epoch 00001: val_matthews_correlation improved from -inf to 0.23477, saving model to weights_2.h5
Epoch 2/50
4789/4789 [==============================] - 2s 462us/step - loss: 0.1776 - matthews_correlation: 0.6545 - val_loss: 0.1639 - val_matthews_correlation: 0.2553

Epoch 00002: val_matthews_correlation improved from 0.23477 to 0.25530, saving model to weights_2.h5
Epoch 3/50
4789/4789 [==============================] - 2s 460us/step - loss: 0.1244 - matthews_correlation: 0.7909 - val_loss: 0.1176 - val_matthews_correlation: 0.2851

Epoch 00003: val_matthews_correlation improved from 0.25530 to 0.28505, saving model to weights_2.h5
Epoch 4/50
4789/4789 [==============================] - 2s 462us/step - loss: 0.0858 - matthews_correlation: 0.8117 - val_


Epoch 00033: val_matthews_correlation did not improve from 0.54325
Epoch 34/50
4789/4789 [==============================] - 2s 463us/step - loss: 0.0402 - matthews_correlation: 0.8902 - val_loss: 0.0539 - val_matthews_correlation: 0.5006

Epoch 00034: val_matthews_correlation did not improve from 0.54325
Epoch 35/50
4789/4789 [==============================] - 2s 462us/step - loss: 0.0408 - matthews_correlation: 0.9083 - val_loss: 0.0614 - val_matthews_correlation: 0.5149

Epoch 00035: val_matthews_correlation did not improve from 0.54325
Epoch 36/50
4789/4789 [==============================] - 2s 463us/step - loss: 0.0432 - matthews_correlation: 0.8910 - val_loss: 0.0681 - val_matthews_correlation: 0.5056

Epoch 00036: val_matthews_correlation did not improve from 0.54325
Epoch 37/50
4789/4789 [==============================] - 2s 459us/step - loss: 0.0430 - matthews_correlation: 0.8871 - val_loss: 0.0663 - val_matthews_correlation: 0.5212

Epoch 00037: val_matthews_correlation did n

4789/4789 [==============================] - 2s 460us/step - loss: 0.0505 - matthews_correlation: 0.8757 - val_loss: 0.0556 - val_matthews_correlation: 0.4213

Epoch 00016: val_matthews_correlation did not improve from 0.45306
Epoch 17/50
4789/4789 [==============================] - 2s 459us/step - loss: 0.0436 - matthews_correlation: 0.9024 - val_loss: 0.0561 - val_matthews_correlation: 0.4505

Epoch 00017: val_matthews_correlation did not improve from 0.45306
Epoch 18/50
4789/4789 [==============================] - 2s 462us/step - loss: 0.0435 - matthews_correlation: 0.9010 - val_loss: 0.0598 - val_matthews_correlation: 0.4109

Epoch 00018: val_matthews_correlation did not improve from 0.45306
Epoch 19/50
4789/4789 [==============================] - 2s 459us/step - loss: 0.0422 - matthews_correlation: 0.8997 - val_loss: 0.0552 - val_matthews_correlation: 0.4187

Epoch 00019: val_matthews_correlation did not improve from 0.45306
Epoch 20/50
4789/4789 [==============================] -

4789/4789 [==============================] - 2s 460us/step - loss: 0.0332 - matthews_correlation: 0.9247 - val_loss: 0.0606 - val_matthews_correlation: 0.4280

Epoch 00050: val_matthews_correlation did not improve from 0.48117
Beginning fold 5
Train on 4790 samples, validate on 1196 samples
Epoch 1/50
4790/4790 [==============================] - 3s 709us/step - loss: 0.3232 - matthews_correlation: 0.1001 - val_loss: 0.2105 - val_matthews_correlation: 0.2005

Epoch 00001: val_matthews_correlation improved from -inf to 0.20053, saving model to weights_4.h5
Epoch 2/50
4790/4790 [==============================] - 2s 459us/step - loss: 0.1775 - matthews_correlation: 0.5392 - val_loss: 0.1163 - val_matthews_correlation: 0.3114

Epoch 00002: val_matthews_correlation improved from 0.20053 to 0.31145, saving model to weights_4.h5
Epoch 3/50
4790/4790 [==============================] - 2s 457us/step - loss: 0.1200 - matthews_correlation: 0.7875 - val_loss: 0.1053 - val_matthews_correlation: 0.27

4790/4790 [==============================] - 2s 460us/step - loss: 0.0481 - matthews_correlation: 0.9053 - val_loss: 0.0714 - val_matthews_correlation: 0.2947

Epoch 00033: val_matthews_correlation did not improve from 0.38733
Epoch 34/50
4790/4790 [==============================] - 2s 455us/step - loss: 0.0499 - matthews_correlation: 0.8925 - val_loss: 0.0591 - val_matthews_correlation: 0.3656

Epoch 00034: val_matthews_correlation did not improve from 0.38733
Epoch 35/50
4790/4790 [==============================] - 2s 457us/step - loss: 0.0430 - matthews_correlation: 0.9052 - val_loss: 0.0542 - val_matthews_correlation: 0.3635

Epoch 00035: val_matthews_correlation did not improve from 0.38733
Epoch 36/50
4790/4790 [==============================] - 2s 459us/step - loss: 0.0417 - matthews_correlation: 0.8965 - val_loss: 0.0559 - val_matthews_correlation: 0.3840

Epoch 00036: val_matthews_correlation did not improve from 0.38733
Epoch 37/50
4790/4790 [==============================] -

((5986,), (5986,))

In [13]:
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    
    y_pred = tf.convert_to_tensor(y_pred, np.float64)
    y_true = tf.convert_to_tensor(y_true, np.float64)
    
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [16]:
optimal_values = threshold_search(y_val, preds_val)
best_threshold = optimal_values['threshold']
best_score = optimal_values['matthews_correlation']

100%|██████████| 100/100 [00:16<00:00,  4.04it/s]


In [17]:
print("""------------------------------
Finished training a LSTM model.
CV scores: %.3f
------------------------------""" % (best_score))

------------------------------
Finished training a LSTM model.
CV scores: 0.909
------------------------------


# Calculate proba of test dataset

In [18]:
meta_test = pd.read_csv('../input/vsb-power-line-fault-detection/metadata_test.csv')
X_test_input = np.load("../input/folk-base-neural-network-using-lstm/X_test.npy")
features_test = np.load("../input/folk-base-neural-network-using-lstm/features_test.npy")
submission = pd.read_csv('../input/vsb-power-line-fault-detection/sample_submission.csv')

In [19]:
preds_test = []
for i in range(N_SPLITS):
    model.load_weights('weights_{}.h5'.format(i))
    pred = model.predict([X_test_input, features_test], batch_size=300, verbose=1)
    pred_3 = []
    for pred_scalar in pred:
        for i in range(3):
            pred_3.append(pred_scalar)
    preds_test.append(pred_3)


6779/6779 [==============================] - 1s 117us/step


In [20]:
optimal_values = threshold_search(y_val, preds_val)
best_threshold = optimal_values['threshold']
best_score = optimal_values['matthews_correlation']


100%|██████████| 100/100 [00:38<00:00,  2.04it/s]


In [21]:
preds_test = (np.squeeze(np.mean(preds_test, axis=0)) > best_threshold).astype(np.int)
preds_test.shape

(20337,)

In [22]:
submission['target'] = preds_test
submission.to_csv('submission.csv', index=False)
submission.head()

,signal_id,target
0,8712,0
1,8713,0
2,8714,0
3,8715,0
4,8716,0


In [23]:
submission[submission.target == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 738 entries, 57 to 20324
Data columns (total 2 columns):
signal_id    738 non-null int64
target       738 non-null int64
dtypes: int64(2)
memory usage: 17.3 KB
